In [1]:
import pandas as pd
from sklearn.metrics import classification_report
import os
import warnings
from configs.data import MACHINE_LEARNING_DATASET_PATH
from machine_learning.utils import split_data, scale_dataset
from machine_learning.neural_networks.fnn import fnn_model, train_fnn_model
from machine_learning.neural_networks.ann import ann_model, train_ann_model
from tensorflow.keras.models import load_model
from configs.enums import Column, RiskClassifications
warnings.simplefilter(action='ignore', category=FutureWarning)

2023-12-05 10:09:45.138036: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-05 10:09:45.160337: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-05 10:09:45.669863: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-05 10:09:45.670072: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-05 10:09:45.670475: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

3.11.6 | packaged by conda-forge | (main, Oct  3 2023, 10:40:35) [GCC 12.3.0]
2.14.0


## 1. Loading the dataset

In [ ]:
df = pd.read_excel(MACHINE_LEARNING_DATASET_PATH)
train, valid, test = split_data(df)
test, x_test, test_labels = scale_dataset(test, oversample=False)

## 2. FNN

In [ ]:
def tune_fnn_model(df, layers, units, dropout_rates, learning_rates):
    least_val_loss = float('inf')
    least_val_loss_params = []
    
    i = 1 
    max = len(layers) * len(units) * len(dropout_rates) * len(learning_rates)
    for l in layers:
        for u in units:
            for dr in dropout_rates:
                for lr in learning_rates:  
                    print(f"[{i}/{max}] Layers: {l}; Units: {u}; Dropout rate: {dr}; Learning rate: {lr};")
                    
                    model, _, _ = train_fnn_model(
                        df, 
                        epochs=100, 
                        patience=20, 
                        layers=l, 
                        units=u,
                        dropout_rate=dr,
                        learning_rate=lr,
                        verbose=0,
                        disable_save=True,
                        disable_plot_history=True,
                        disable_print_report=True)
                    
                    val_loss, val_acc = model.evaluate(x_test, test_labels)
                    print(f"Loss: {val_loss}; Accuracy: {val_acc};")
                    if val_loss < least_val_loss:
                        model.save(os.path.join(os.environ["OUTPUT_PATH"], "Risk_factor_dnn_model.keras"))
                        least_val_loss = val_loss
                        least_val_loss_params = [l, u, dr, lr]
                        
                    i += 1
                        
    print(least_val_loss_params)
    print(least_val_loss)

### 2.1 Tuning

In [ ]:
tune_fnn_model(
        df=df,
        layers=[1], # 1, 2, 3, 4, 5, 6
        units=[192, 224, 256], # 8, 16, 32, 64, 96, 128, 160, 192
        dropout_rates=[0.2], # 0.2, 0.3, 0.4, 0.5
        learning_rates=[0.001, 0.00125, 0.0015, 0.00175, 0.002], # 0.0001, 0.0005, 0.00075, 0.001, 0.00125, 0.0015, 0.00175, 0.002  
    )

### 2.2 Loading from a file

In [ ]:
model_file = "Risk_factor_fnn_model.keras" # "Adam_6_192_0.2_0.00175_0.1515021026134491_100.keras" # "Risk_factor_dnn_model.keras"
model = load_model(os.path.join(os.environ["OUTPUT_PATH"], model_file))

y_pred = model.predict(x_test).argmax(axis=1)
print(classification_report(test_labels, y_pred))

### 2.3 Manual tuning

In [ ]:
model, _, _ = train_fnn_model(
                        df, 
                        epochs=100, 
                        patience=20, 
                        layers=6, 
                        units=192,
                        dropout_rate=0.2,
                        learning_rate=0.00175)

### 2.4 Shap

In [14]:
import sys
python_version = sys.version.split("|")[0].rstrip()

if python_version == "3.11.6":
    import shap
    print("Shap imported.")
else:
    print("Shap could not be installed on this docker image ('tensorflow-2150'), build 'tensorflow-2140' from docker compose.")

Shap imported.


In [ ]:
def shapify(data: pd.DataFrame):
    train, valid, test = split_data(data)

    train, x_train, train_labels = scale_dataset(train, oversample=True)
    valid, x_val, val_labels = scale_dataset(valid, oversample=False)
    test, x_test, test_labels = scale_dataset(test, oversample=False)
    
    explainer = shap.KernelExplainer(model.predict, x_train)
    shap_values = explainer.shap_values(x_test, nsamples=100) # nsamples { default = 2 * X.shape[1] + 2048 = 2066 }
    # explainer.save()
    
    return explainer, shap_values, x_test

In [ ]:
explainer, shap_values, shap_x_test = shapify(df[:1000])
shap.summary_plot(shap_values, shap_x_test, feature_names=df.columns[Column.COUNTRY_RISK], class_names=RiskClassifications.get_names())

## 3. Artificial Neural Network (ANN)

In [ ]:
def tune_ann_model(df, units, dropout_rates, learning_rates):
    least_val_loss = float('inf')
    least_val_loss_params = []
    
    i = 1 
    max = len(units) * len(dropout_rates) * len(learning_rates)
    for u in units:
        for dr in dropout_rates:
            for lr in learning_rates:  
                print(f"[{i}/{max}] Units: {u}; Dropout rate: {dr}; Learning rate: {lr};")
                
                model, _, _ = train_ann_model(
                    df, 
                    epochs=100, 
                    patience=20, 
                    units=u,
                    dropout_rate=dr,
                    learning_rate=lr,
                    verbose=0,
                    disable_save=True,
                    disable_plot_history=True,
                    disable_print_report=True)
                
                val_loss, val_acc = model.evaluate(x_test, test_labels)
                print(f"Loss: {val_loss}; Accuracy: {val_acc};")
                if val_loss < least_val_loss:
                    model.save(os.path.join(os.environ["OUTPUT_PATH"], "Risk_factor_ann_model.keras"))
                    least_val_loss = val_loss
                    least_val_loss_params = [u, dr, lr]
                    
                i += 1
                        
    print(least_val_loss_params)
    print(least_val_loss)

### 3.1 Tuning

In [ ]:
tune_ann_model(df=df,
                units=[320],
                dropout_rates=[0.2],
                learning_rates=[0.002])

In [ ]:
### 3.2 Manual tuning

In [ ]:
model, _, _ = train_ann_model(
                        df, 
                        epochs=100, 
                        patience=10, 
                        units=320,
                        dropout_rate=0.2,
                        learning_rate=0.002,
                        verbose=2,
                        disable_save=True)

val_loss, val_acc = model.evaluate(x_test, test_labels)
print(f"Loss: {val_loss}; Accuracy: {val_acc};")